### 단어를 시퀀스로 처리하기: 시퀀스 모델 방식
- 기존에는 bigram과 같이 순서 기반의 특성을 수동으로 만들어 학습함
- 그럼 raw text sequence를 모델에 전달하여 스스로 이런 특성을 학습하도록 하면?? -> **Sequence model**

**Sequence model**  
1. 입력 샘플을 정수 인덱스의 시퀀스로 표현 (하나의 정수가 하나의 단어)
2. 각 정수를 벡터로 매핑하여 벡터 시퀀스를 얻음
3. 해당 벡터 시퀀스를 1D ConvNet, RNN, transformer와 같이 인접한 벡터의 특징을 비교할 수 있는 층에 전달

**데이터 다운로드**

In [1]:
!rm -r aclImdb
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup

rm: cannot remove 'aclImdb': No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  10.6M      0  0:00:07  0:00:07 --:--:-- 11.7M


**데이터 준비**


In [2]:
import os, pathlib, shutil, random
from tensorflow import keras

batch_size = 32
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


#### 첫 번째 예제

**정수 시퀀스 데이터셋 준비하기**

In [3]:
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    # 적당한 입력 크기를 유지하기 위해 입력에서 600개 단어 이후는 잘라버림
    # 평균 리뷰 길이가 233개의 단어고 600개의 단어보다 긴 리뷰는 5%뿐 ㄱㅊ
    output_sequence_length=max_length,
)

text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

**원-핫 인코딩된 벡터 시퀀스로 시퀀스 모델 만들기**  
- 각 차원은 어휘 사전에 있는 하나의 단어를 표현

In [6]:
import tensorflow as tf

class EmbeddedLayer(keras.Layer):
    def call(self, x):
        return tf.one_hot(x, depth=max_tokens)

# 정수 시퀀스
inputs = keras.Input(shape=(None,), dtype="int64")
# 정수를 200000 차원의 이진 벡터로 인코딩
embedded = EmbeddedLayer()(inputs)
# embedded = tf.one_hot(inputs, depth=max_tokens)
# ValueError: A KerasTensor cannot be used as input to a TensorFlow function.
# A KerasTensor is a symbolic placeholder for a shape and dtype, used when
# constructing Keras Functional models or Keras Functions. You can only use it
# as input to a Keras layer or a Keras operation (from the namespaces
# `keras.layers` and `keras.operations`). You are likely doing something like:

# 양방향 LSTM 추가
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
# 분류층 추가
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedded_layer (EmbeddedLayer)       │ (None, None, 20000)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 64)                  │       5,128,448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,128,513 (19.56 MB)

 Trainable params: 5,128,513 (19.56 MB)

 Non-trainable params: 0 (0.00 B)

**첫 번째 시퀀스 모델 훈련하기**

In [7]:
callbacks = [
    keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.h5",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("one_hot_bidir_lstm.h5")
print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step - accuracy: 0.6275 - loss: 0.6314

625/625 ━━━━━━━━━━━━━━━━━━━━ 227s 356ms/step - accuracy: 0.6276 - loss: 0.6313 - val_accuracy: 0.8596 - val_loss: 0.3672
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step - accuracy: 0.8600 - loss: 0.3737

625/625 ━━━━━━━━━━━━━━━━━━━━ 260s 356ms/step - accuracy: 0.8600 - loss: 0.3736 - val_accuracy: 0.8632 - val_loss: 0.3480
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step - accuracy: 0.8920 - loss: 0.3046

625/625 ━━━━━━━━━━━━━━━━━━━━ 262s 356ms/step - accuracy: 0.8920 - loss: 0.3046 - val_accuracy: 0.8746 - val_loss: 0.3040
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 240s 384ms/step - accuracy: 0.9097 - loss: 0.2622 - val_accuracy: 0.8446 - val_loss: 0.3525
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step - accuracy: 0.9247 - loss: 0.2272

625/625 ━━━━━━━━━━━━━━━━━━━━ 245s 356ms/step - accuracy: 0.9247 - loss: 0.2272 - val_accuracy: 0.8906 - val_loss: 0.2837
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 262s 356ms/step - accuracy: 0.9343 - loss: 0.2011 - val_accuracy: 0.8820 - val_loss: 0.3234
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 240s 385ms/step - accuracy: 0.9390 - loss: 0.1848 - val_accuracy: 0.8844 - val_loss: 0.2961
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 263s 386ms/step - accuracy: 0.9462 - loss: 0.1604 - val_accuracy: 0.8838 - val_loss: 0.3432
Epoch 9/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 244s 358ms/step - accuracy: 0.9575 - loss: 0.1382 - val_accuracy: 0.8780 - val_loss: 0.3987
Epoch 10/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 263s 360ms/step - accuracy: 0.9596 - loss: 0.1298 - val_accuracy: 0.8810 - val_loss: 0.4225


ValueError: Unknown layer: 'EmbeddedLayer'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

- 해당 모델 훈련이 매우 느림 : 입력 크기(600, 20000)가 크기 때문
  - 하나의 영화 리뷰는 1200만 개의 부동 소수점임
  - 따라서 양방향 LSTM이 해야 할 일이 많음
- 테스트 정확도 : 87%

**딱히 성능이 좋아보이진 않네**  

#### 단어 임베딩 이해하기 **(word embedding)**
1. 원핫인코딩
   - 희소하고 고차원인 이진 벡터를 생성
   - 하드코딩
   - 일종의 특성 공학을 수행한 것
   - 특성 공간의 구조에 대한 기초적인 가정을 모델에 주입한 것
     - 가정: 인코딩하는 토큰은 서로 독립적이라는 것
   - 원-핫 벡터는 서로 모두 직교
   - 하지만 단어의 경우, 이 가정은 명백히 잘못되었음

2. **단어 임베딩**
  - **저차원**의 부동 소수점 벡터 (밀집 벡터)
  - **데이터로부터 학습**
  - 많은 정보를 더 적은 차원으로 압축
  - 사람의 언어를 구조적인 기하학적 공간에 매핑
  - 단어는 구조적인 공간을 형성 -> 단어에 공유되는 정보가 있음
  - 예를 들어, "movie"와 "film"은 대부분 동일한 의미로 사용되기 때문에 직교해서는 안됨
  - 두 단어 벡터 사이의 기하학적 관계는 단어 사이의 의미 관계를 반영해야 함
  - 예를 들어, 합리적인 단어 벡터 공간에서는 동의어가 비슷한 단어 벡터로 임베딩될 것이라 기대 가능
  - 이런 공간에서는 일반적으로 두 단어 벡터 사이의 기하학적 거리가 단어 사이의 '의미 거리'에 연관되어 있다 할 수 있음
    - 다른 의미를 가지는 단어는 서로 멀고 관련이 있는 단어는 가까이 놓여 있어야 함
  - 실제 단어 임베딩 공간에서 의미 있는 기하학적 변환의 일반적인 예는 '성별' 벡터 & '복수(plural)' 벡터
    - 'king' vector + 'female' vector = 'queen' vector
    - 'king' vector + 'plural' vector = 'kings' vector
  - 단어 임베딩 만드는 방법
    - 현재 작업과 함께 단어 임베딩 학습
    - 사전 훈련된 단어 임베딩 사용

#### 임베딩 층으로 단어 임베딩 학습하기
- 문제에 따라 다른 공간이 필요
- 따라서 새로운 작업에는 새로운 임베딩 학습 필요

**`Embedding` 층 만들기**

In [8]:
# embedding layer의 가중치 학습
# embedding layer는 적어도 2개의 매개변수 필요
embedding_layer = layers.Embedding(input_dim=max_tokens, output_dim=256)

**Embedding layer**  
- 정수 인덱스를 밀집 벡터로 매핑하는 딕셔너리로 이해  
- 정수를 입력으로 받아 내부 딕셔너리에서 이 정수에 연관된 벡터를 찾아 봔환
- Input : 크기가 (batch_size, sequence_length)인 rank-2 intager tensor
- Output : 크기가 (batch_size, sequence_length, embedding_dimensionality) rank-3 float tensor

**밑바닥부터 훈련하는 `Embedding` 층을 사용한 모델**

In [9]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_lstm.h5",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_lstm.h5")
print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_1 (Embedding)              │ (None, None, 256)           │       5,120,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 64)                  │          73,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,194,049 (19.81 MB)

 Trainable params: 5,194,049 (19.81 MB)

 Non-trainable params: 0 (0.00 B)

...

625/625 ━━━━━━━━━━━━━━━━━━━━ 37s 55ms/step - accuracy: 0.6489 - loss: 0.6170 - val_accuracy: 0.8416 - val_loss: 0.3846
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.8329 - loss: 0.4115

625/625 ━━━━━━━━━━━━━━━━━━━━ 38s 51ms/step - accuracy: 0.8329 - loss: 0.4114 - val_accuracy: 0.8658 - val_loss: 0.3368
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.8688 - loss: 0.3493

625/625 ━━━━━━━━━━━━━━━━━━━━ 40s 50ms/step - accuracy: 0.8688 - loss: 0.3493 - val_accuracy: 0.8758 - val_loss: 0.3161
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 39s 46ms/step - accuracy: 0.8918 - loss: 0.2947 - val_accuracy: 0.8700 - val_loss: 0.3195
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.9140 - loss: 0.2472

625/625 ━━━━━━━━━━━━━━━━━━━━ 41s 47ms/step - accuracy: 0.9140 - loss: 0.2472 - val_accuracy: 0.8756 - val_loss: 0.3127
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 40s 46ms/step - accuracy: 0.9308 - loss: 0.2057 - val_accuracy: 0.8836 - val_loss: 0.3434
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 31s 50ms/step - accuracy: 0.9369 - loss: 0.1887 - val_accuracy: 0.8666 - val_loss: 0.4555
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 46ms/step - accuracy: 0.9539 - loss: 0.1449 - val_accuracy: 0.8812 - val_loss: 0.4045
Epoch 9/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 46ms/step - accuracy: 0.9621 - loss: 0.1258 - val_accuracy: 0.8810 - val_loss: 0.4139
Epoch 10/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 46ms/step - accuracy: 0.9711 - loss: 0.0979 - val_accuracy: 0.8824 - val_loss: 0.4552


782/782 ━━━━━━━━━━━━━━━━━━━━ 17s 20ms/step - accuracy: 0.8634 - loss: 0.3426
테스트 정확도: 0.857


256차원 벡터를 처리하기 때문에 원-핫 모델보다 훨씬 빠름  
테스트 정확도는 비슷  
약간 적은 데이터를 사용하기 때문이겠지? 600개 단어 이후는 모두 잘라버리니깐..

#### 패딩과 마스킹 이해하기
- TextVectorization layer에 `output_sequence_length=max_length` 옵션을 사용했기 때문에 입력 시퀀스가 0으로 가득 차 있으면 모델의 성능에 나쁜 영향을 미침
- 600개의 토큰보다 짧은 문장은 600개의 토큰이 되도록 끝에 0을 채움
- RNN의 내부 상태에 저장된 정보는 의미 없는 입력을 처리하면서 점차 사라질 것
- RNN 층이 이런 패딩을 건너뛰도록 해야해! : Masking

**Masking**  
- 1과 0(혹은 True/False)으로 이루어진 (batch_size, sequence_length) 크기의 텐서
- `mask[i, t]` 원소는 샘플 i의 타임스텝 t를 건너뛰어야 할지(0, False) 처리해야 할지(1, True) 나타냄
- 활성화 방법 : `mask_zero=True` 설정

In [12]:
emdedding_layer = layers.Embedding(input_dim=10, output_dim=256, mask_zero=True)
some_input = [
    [4, 3, 2, 1, 0, 0, 0],
    [5, 4, 3, 2, 1, 0, 0],
    [2, 1, 0, 0, 0, 0, 0]
]
mask = embedding_layer.compute_mask(some_input)
mask

**마스킹을 활성화한 `Embedding` 층 사용하기**  
- 케라스가 마스킹을 처리할 수 있는 모든 층에 시퀀스에 부착된 metadata의 일부로 자동으로 전달
- 손실 함수 역시 마스킹을 사용하여 출력 시퀀스에서 마스킹된 스텝을 건너뜀

In [14]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(
    input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_lstm_with_masking.h5",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_lstm_with_masking.h5")
print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_3 (Embedding)   │ (None, None, 256)      │      5,120,000 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal (NotEqual)      │ (None, None)           │              0 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_2           │ (None, 64)             │         73,984 │ embedding_3[0][0],     │
│ (Bidirectional)           │                        │                │ not_equal[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 64)             │              0 │ bidirectional_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 1)              │             65 │ dropout_2[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 5,194,049 (19.81 MB)

 Trainable params: 5,194,049 (19.81 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.6932 - loss: 0.5603

625/625 ━━━━━━━━━━━━━━━━━━━━ 32s 47ms/step - accuracy: 0.6934 - loss: 0.5602 - val_accuracy: 0.8622 - val_loss: 0.3288
Epoch 2/10
624/625 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.8697 - loss: 0.3183

625/625 ━━━━━━━━━━━━━━━━━━━━ 32s 51ms/step - accuracy: 0.8697 - loss: 0.3182 - val_accuracy: 0.8738 - val_loss: 0.3020
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 39s 48ms/step - accuracy: 0.9025 - loss: 0.2454 - val_accuracy: 0.8642 - val_loss: 0.3345
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 46ms/step - accuracy: 0.9239 - loss: 0.2035 - val_accuracy: 0.8850 - val_loss: 0.3121
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 45s 53ms/step - accuracy: 0.9442 - loss: 0.1551 - val_accuracy: 0.8694 - val_loss: 0.3506
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 41s 53ms/step - accuracy: 0.9560 - loss: 0.1216 - val_accuracy: 0.8800 - val_loss: 0.3435
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 42s 55ms/step - accuracy: 0.9678 - loss: 0.0908 - val_accuracy: 0.8712 - val_loss: 0.3945
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 47ms/step - accuracy: 0.9795 - loss: 0.0653 - val_accuracy: 0.8674 - val_loss: 0.4059
Epoch 9/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 32s 51ms/step - accuracy: 0.9844 - loss: 0.0489 - val_accurac

ValueError: Unknown layer: 'NotEqual'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

#### 사전 훈련된 단어 임베딩 사용하기
- 미리 계산된 임베딩 공간의 임베딩 벡터를 로드해서 사용
- 충분한 데이터가 없어 학습하기 힘들지만 꽤 일반적인 특성이 필요할 때 사용
- 단어 임베딩 기법
  - Word2vec : 성별처럼 구체적인 의미가 있는 속성을 잡아 냄
  - GloVe : 단어의 동시 출현(co-occurrence) 통계를 기록한 행렬을 분해하는 기법 사용

In [15]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2025-02-04 05:58:02--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-02-04 05:58:02--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-02-04 05:58:02--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

**GloVe 단어 임베딩 파일 파싱하기**

In [16]:
import numpy as np
path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print(f"단어 벡터 개수: {len(embeddings_index)}")

단어 벡터 개수: 400000


**GloVe 단어 임베딩 행렬 준비하기**

In [17]:
embedding_dim = 100

# 인덱싱된 단어를 추출
vocabulary = text_vectorization.get_vocabulary()
# 어휘 사전에 있는 단어와 인덱스 매핑
word_index = dict(zip(vocabulary, range(len(vocabulary))))

# GloVe 벡터로 채울 행렬 준비
embedding_matrix = np.zeros((max_tokens, embedding_dim))
for word, i in word_index.items():
    if i < max_tokens:
        embedding_vector = embeddings_index.get(word)
    # 인덱스 i에 대한 단어 벡터로 행렬의 i번째 항목을 채움
    # 임베딩 인덱스에 없는 단어는 모두 0이 됨
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [18]:
# Constant 초기화 : Embedding 층에 사전 훈련된 임베딩 로드
embedding_layer = layers.Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    # 훈련동안 사전 훈련된 표현이 변경되지 않도록 trainable=False
    trainable=False,
    mask_zero=True,
)

**사전 훈련된 임베딩을 사용하는 모델**

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("glove_embeddings_sequence_model.h5",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("glove_embeddings_sequence_model.h5")
print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_3 (Embedding)     (None, None, 100)         2000000   
                                                                 
 bidirectional_3 (Bidirecti  (None, 64)                34048     
 onal)                                                           
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2034113 (7.76 MB)
Trainable params: 34113 (133.25 KB)
Non-trainable params: 2000000 (7.63 MB)
_________________

이것도 딱히 도움되는것 같지는 않네